Tests to run GenIce on Jupyter and Google Colaboratory.

In [2]:
%pip install genice

You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from genice import genice as gi
from genice.importer import safe_import

lattice_maker = safe_import("lattice", "3")
formatter     = safe_import("format", "gromacs")
water         = safe_import("molecule", "4site")
guests        = None
lat = gi.GenIce(lattice_maker)
ice = lat.generate_ice(water, formatter=formatter)
print(ice)

Generated by GenIce https://github.com/vitroid/GenIce 
48
    1ICE     OW    1   0.258   0.266   0.013
    1ICE    HW1    2   0.354   0.265   0.019
    1ICE    HW2    3   0.238   0.202  -0.055
    1ICE     MW    4   0.268   0.258   0.005
    2ICE     OW    5   0.410   0.129   0.322
    2ICE    HW1    6   0.340   0.103   0.382
    2ICE    HW2    7   0.414   0.224   0.328
    2ICE     MW    8   0.402   0.138   0.330
    3ICE     OW    9   0.597   0.462   0.558
    3ICE    HW1   10   0.601   0.558   0.552
    3ICE    HW2   11   0.534   0.437   0.491
    3ICE     MW   12   0.590   0.471   0.549
    4ICE     OW   13   0.138   0.410   0.378
    4ICE    HW1   14   0.063   0.425   0.435
    4ICE    HW2   15   0.111   0.339   0.321
    4ICE     MW   16   0.125   0.403   0.378
    5ICE     OW   17   0.462   0.591   0.146
    5ICE    HW1   18   0.557   0.588   0.152
    5ICE    HW2   19   0.437   0.661   0.206
    5ICE     MW   20   0.471   0.599   0.154
    6ICE     OW   21   0.409   0.401   0.3